<c><h1>Data Cleaning: Diabetes Control Efficacy Trial (DiaCET-2024)</h1></c>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_excel('Diabetes_Dataset.xlsx')
data.head()

,ParticipantID,TreatmentGroup,RandomizationDate,StudySite,Age,Gender,BMI,DiabetesDuration,SmokingStatus,EducationLevel,...,WeightChange,FollowupWeight,QoLChange,FollowupQoL,TargetAchieved,PatientSatisfaction,AdverseEvents,DaysToComplication,ComplicationOccurred,MedicationAdherence
0,DM1001,0,2024-04-26,0,40,1,23.5,18,2,0,...,0.2,113.0,6.3,50.0,0.0,1.0,0,149,1,96.7
1,DM1002,0,2024-07-12,3,32,1,36.6,1,0,1,...,-7.5,101.0,1.5,58.0,1.0,3.0,1,94,1,79.8
2,DM1003,0,2024-04-16,3,62,1,31.9,2,0,2,...,-1.3,99.3,7.6,89.0,0.0,0.0,0,215,1,87.7
3,DM1004,0,2024-01-29,1,36,1,35.8,1,1,1,...,-1.1,96.4,-2.6,83.0,0.0,2.0,1,167,1,78.9
4,DM1005,0,2024-04-30,3,48,0,30.7,1,0,1,...,NaN,80.1,10.0,88.0,0.0,2.0,0,365,0,95.5


<h2>1. Basic Data Overview</h2>

In [3]:
print("DATASET SHAPE:")
print("Number of rows:", data.shape[0])
print("Number of columns:", data.shape[1])

DATASET SHAPE:
Number of rows: 300
Number of columns: 30


In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ParticipantID         300 non-null    object        
 1   TreatmentGroup        300 non-null    int64         
 2   RandomizationDate     300 non-null    datetime64[ns]
 3   StudySite             300 non-null    int64         
 4   Age                   300 non-null    int64         
 5   Gender                300 non-null    int64         
 6   BMI                   300 non-null    float64       
 7   DiabetesDuration      300 non-null    int64         
 8   SmokingStatus         300 non-null    int64         
 9   EducationLevel        300 non-null    int64         
 10  Complications         300 non-null    int64         
 11  BaselineHbA1c         300 non-null    float64       
 12  BaselineFBS           288 non-null    float64       
 13  BaselineWeight      

In [5]:
print("Number of datatypes:")
data.dtypes.value_counts()

Number of datatypes:


float64           15
int64             13
object             1
datetime64[ns]     1
Name: count, dtype: int64

<h2>2. Check Data Quality</h2>

<h3>a. Number of missing values</h3>

In [6]:
data.isna().sum()

ParticipantID            0
TreatmentGroup           0
RandomizationDate        0
StudySite                0
Age                      0
Gender                   0
BMI                      0
DiabetesDuration         0
SmokingStatus            0
EducationLevel           0
Complications            0
BaselineHbA1c            0
BaselineFBS             12
BaselineWeight           0
BaselineQoL              0
FollowupCompleted        0
HbA1cChange             36
FollowupHbA1c           36
FBSChange               36
FollowupFBS             36
WeightChange            36
FollowupWeight          36
QoLChange               36
FollowupQoL             36
TargetAchieved          36
PatientSatisfaction     36
AdverseEvents            0
DaysToComplication       0
ComplicationOccurred     0
MedicationAdherence     24
dtype: int64

<h3>b. Duplicate Records</h3>

In [7]:
data.duplicated().sum()

np.int64(0)

<h3>c. Unique values per column</h3>

In [8]:
data.nunique()

ParticipantID           300
TreatmentGroup            2
RandomizationDate       154
StudySite                 4
Age                      50
Gender                    2
BMI                     156
DiabetesDuration         19
SmokingStatus             3
EducationLevel            3
Complications             4
BaselineHbA1c            46
BaselineFBS             119
BaselineWeight          230
BaselineQoL              60
FollowupCompleted         2
HbA1cChange              48
FollowupHbA1c            77
FBSChange               110
FollowupFBS             141
WeightChange             92
FollowupWeight          220
QoLChange               169
FollowupQoL              75
TargetAchieved            2
PatientSatisfaction       4
AdverseEvents             4
DaysToComplication      142
ComplicationOccurred      2
MedicationAdherence     179
dtype: int64

<h2>3. Data Cleaning</h2>

In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

In [10]:
df=data.copy()

<h3>a. Missing value handling by feature engineering

In [11]:
#fill only where baseline is missing and both followup and change are present
mask = df["BaselineFBS"].isna() & df["FollowupFBS"].notna() & df["FBSChange"].notna()
df.loc[mask, "BaselineFBS"] = df.loc[mask, "FollowupFBS"] - df.loc[mask, "FBSChange"]

In [12]:
followup_cols = ['FollowupHbA1c','FollowupFBS','FollowupWeight','FollowupQoL']
change_cols = ['HbA1cChange','FBSChange','WeightChange','QoLChange']
baseline_cols = ['BaselineHbA1c','BaselineFBS','BaselineWeight','BaselineQoL']

In [13]:
#fill only where followup is missing and both baseline and change are present
for col_b, col_f, col_c in zip(baseline_cols, followup_cols, change_cols):
    mask = df[col_f].isna() & df[col_b].notna() & df[col_c].notna()
    df.loc[mask, col_f] = df.loc[mask, col_b] + df.loc[mask, col_c]

In [14]:
#fill only where change is missing and both baseline and followup are present
for col_b, col_f, col_c in zip(baseline_cols, followup_cols, change_cols):
    mask = df[col_c].isna() & df[col_b].notna() & df[col_f].notna()
    df.loc[mask, col_c] = df.loc[mask, col_f] - df.loc[mask, col_b]

In [15]:
# Checking the missing values after feature engineering
compare = pd.DataFrame({
    'Before Feature Engineering': data.isna().sum(),
    'After Feature Engineering': df.isna().sum()
})
compare

,Before Feature Engineering,After Feature Engineering
ParticipantID,0,0
TreatmentGroup,0,0
RandomizationDate,0,0
StudySite,0,0
Age,0,0
Gender,0,0
BMI,0,0
DiabetesDuration,0,0
SmokingStatus,0,0
EducationLevel,0,0


<h3>b. Missing value imputation

In [16]:
# Imputing the remaining missing values
knn = KNNImputer(n_neighbors=5)
cols_to_be_imputed = ['BaselineFBS','HbA1cChange','FollowupHbA1c','FBSChange','FollowupFBS','WeightChange','FollowupWeight',
                      'QoLChange','FollowupQoL','MedicationAdherence']
df[cols_to_be_imputed] = knn.fit_transform(df[cols_to_be_imputed]).round(1)

mice = IterativeImputer(max_iter = 100 ,random_state=42)
cols_to_be_imputed_mice = ['TargetAchieved','PatientSatisfaction']
df[cols_to_be_imputed_mice] = mice.fit_transform(df[cols_to_be_imputed_mice]).round().astype(int)

df.isna().sum()

ParticipantID           0
TreatmentGroup          0
RandomizationDate       0
StudySite               0
Age                     0
Gender                  0
BMI                     0
DiabetesDuration        0
SmokingStatus           0
EducationLevel          0
Complications           0
BaselineHbA1c           0
BaselineFBS             0
BaselineWeight          0
BaselineQoL             0
FollowupCompleted       0
HbA1cChange             0
FollowupHbA1c           0
FBSChange               0
FollowupFBS             0
WeightChange            0
FollowupWeight          0
QoLChange               0
FollowupQoL             0
TargetAchieved          0
PatientSatisfaction     0
AdverseEvents           0
DaysToComplication      0
ComplicationOccurred    0
MedicationAdherence     0
dtype: int64

<h3>c. Changing the Datatypes

In [17]:
#changing the datatype of categorical columns
for col in df.columns[df.nunique()<5]:
    df[col] = df[col].astype('category')
df.dtypes.value_counts()

float64           13
category           5
category           4
int64              4
category           2
object             1
datetime64[ns]     1
Name: count, dtype: int64

In [18]:
# Check properties of each categorical column
cat_cols = df.select_dtypes(include='category').columns
for col in cat_cols:
    print(f"\n{col}:")
    print(f"  Ordered: {df[col].cat.ordered}")
    print(f"  Categories: {df[col].cat.categories.tolist()}")
    print(f"  Dtype: {df[col].dtype}")


TreatmentGroup:
  Ordered: False
  Categories: [0, 1]
  Dtype: category

StudySite:
  Ordered: False
  Categories: [0, 1, 2, 3]
  Dtype: category

Gender:
  Ordered: False
  Categories: [0, 1]
  Dtype: category

SmokingStatus:
  Ordered: False
  Categories: [0, 1, 2]
  Dtype: category

EducationLevel:
  Ordered: False
  Categories: [0, 1, 2]
  Dtype: category

Complications:
  Ordered: False
  Categories: [0, 1, 2, 3]
  Dtype: category

FollowupCompleted:
  Ordered: False
  Categories: [0, 1]
  Dtype: category

TargetAchieved:
  Ordered: False
  Categories: [0, 1]
  Dtype: category

PatientSatisfaction:
  Ordered: False
  Categories: [0, 1, 2, 3]
  Dtype: category

AdverseEvents:
  Ordered: False
  Categories: [0, 1, 2, 3]
  Dtype: category

ComplicationOccurred:
  Ordered: False
  Categories: [0, 1]
  Dtype: category


<h2>4. Saving the cleaned dataset

In [19]:
df.to_csv('/Users/dr.shivambhatnagar/Desktop/github_python/SAP/Diabetes_Dataset_Cleaned.csv', index=False)